In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from store import *
import kbech32

In [2]:
import hashlib
import struct

# Relevant hashing functions as implemented by the golang 
# and rust kaspa codebases
# Note: uses only standart hash libs

def transaction_id(t):
    hasher = hashlib.blake2b(digest_size=32, key=b"TransactionID")
    hasher.update(struct.pack(f"<HQ", t.version, len(t.inputs)))
    for ti in t.inputs:
        hasher.update(ti.previousOutpoint.transactionID.transactionId)
        hasher.update(struct.pack(f"<IQQ", 
                                  ti.previousOutpoint.index, 
                                  0, 
                                  ti.sequence))

    hasher.update(struct.pack(f"<Q", len(t.outputs)))
    for to in t.outputs:
        hasher.update(struct.pack(f"<QHQ", to.value, 
                                  to.scriptPublicKey.version, 
                                  len(to.scriptPublicKey.script)))
        hasher.update(to.scriptPublicKey.script)
        
    hasher.update(struct.pack(f"<Q", t.lockTime))
    hasher.update(t.subnetworkID.subnetworkId)
    hasher.update(struct.pack(f"<QQ", t.gas, len(t.payload)))
    hasher.update(t.payload)
    return hasher.digest()

def transaction_hash(t):
    hasher = hashlib.blake2b(digest_size=32, key=b"TransactionHash")
    hasher.update(struct.pack(f"<HQ", t.version, len(t.inputs)))
    for ti in t.inputs:
        hasher.update(ti.previousOutpoint.transactionID.transactionId)
        hasher.update(struct.pack(f"<IQ", ti.previousOutpoint.index, 
                                  len(ti.signatureScript)))
        hasher.update(ti.signatureScript)
        # Note: a subsequent HF added sig_op_count hashing here
        hasher.update(struct.pack(f"<Q", ti.sequence))

    hasher.update(struct.pack(f"<Q", len(t.outputs)))
    for to in t.outputs:
        hasher.update(struct.pack(f"<QHQ", to.value, 
                                  to.scriptPublicKey.version, 
                                  len(to.scriptPublicKey.script)))
        hasher.update(to.scriptPublicKey.script)
        
    hasher.update(struct.pack(f"<Q", t.lockTime))
    hasher.update(t.subnetworkID.subnetworkId)
    hasher.update(struct.pack(f"<QQ", t.gas, len(t.payload)))
    hasher.update(t.payload)
    return hasher.digest()

def header_hash(h):
    hasher = hashlib.blake2b(digest_size=32, key=b"BlockHash")
    hasher.update(struct.pack(f"<HQ", h.version, len(h.parents)))
    for level_parents in h.parents:
        hasher.update(struct.pack(f"<Q", len(level_parents.parentHashes)))
        for parent in level_parents.parentHashes:
            hasher.update(parent.hash)
    hasher.update(h.hashMerkleRoot.hash)    
    hasher.update(h.acceptedIDMerkleRoot.hash)
    hasher.update(h.utxoCommitment.hash)
    hasher.update(struct.pack(f"<QIQQQQ", 
                              h.timeInMilliseconds, 
                              h.bits, 
                              h.nonce, 
                              h.daaScore, 
                              h.blueScore, 
                              len(h.blueWork)))
    hasher.update(h.blueWork)
    hasher.update(h.pruningPoint.hash)
    return hasher.digest()

In [3]:
# Asserts a verified chaih of hashes from the givan block to the given genesis
def assert_cryptographic_hash_chain_to_genesis(
    store, 
    block_hash, 
    genesis_hash):
    
    i = 0
    while True:
        if block_hash == genesis_hash:
            print('Reached the queried genesis block: \n', 
                  genesis_hash.hex(), 'via', i, 'pruning points')
            return
        header = store.get_raw_header(block_hash)
        # Assert the block hash is correct
        assert(header_hash(header) == block_hash)
        block_hash = header.pruningPoint.hash
        i += 1

In [4]:
# Take genesis hash from current go code
# Golang ref: 
# https://github.com/kaspanet/kaspad/blob/master/domain/dagconfig/genesis.go#L56

genesis_hash = bytes([
    0x58, 0xc2, 0xd4, 0x19, 0x9e, 0x21, 0xf9, 0x10, 
    0xd1, 0x57, 0x1d, 0x11, 0x49, 0x69, 0xce, 0xce, 
    0xf4, 0x8f, 0x9, 0xf9, 0x34, 0xd4, 0x2c, 0xcb, 
    0x6a, 0x28, 0x1a, 0x15, 0x86, 0x8f, 0x29, 0x99])


In [5]:
# Build genesis's coinbase tx payload, which references the pre-halt checkpoint 
# Golang ref: 
# https://github.com/kaspanet/kaspad/blob/master/domain/dagconfig/genesis.go#L18

genesis_tx_payload = bytes([
    0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, # Blue score
    0x00, 0xE1, 0xF5, 0x05, 0x00, 0x00, 0x00, 0x00, # Subsidy
    0x00, 0x00, # Script version
    0x01,                                           # Varint
    0x00,                                           # OP-FALSE
    
    # ומה די עליך ועל אחיך ייטב בשאר כספא ודהבה למעבד כרעות אלהכם תעבדון     
    0xd7, 0x95, 0xd7, 0x9e, 0xd7, 0x94, 0x20, 0xd7,
    0x93, 0xd7, 0x99, 0x20, 0xd7, 0xa2, 0xd7, 0x9c,
    0xd7, 0x99, 0xd7, 0x9a, 0x20, 0xd7, 0x95, 0xd7,
    0xa2, 0xd7, 0x9c, 0x20, 0xd7, 0x90, 0xd7, 0x97,
    0xd7, 0x99, 0xd7, 0x9a, 0x20, 0xd7, 0x99, 0xd7,
    0x99, 0xd7, 0x98, 0xd7, 0x91, 0x20, 0xd7, 0x91,
    0xd7, 0xa9, 0xd7, 0x90, 0xd7, 0xa8, 0x20, 0xd7,
    0x9b, 0xd7, 0xa1, 0xd7, 0xa4, 0xd7, 0x90, 0x20,
    0xd7, 0x95, 0xd7, 0x93, 0xd7, 0x94, 0xd7, 0x91,
    0xd7, 0x94, 0x20, 0xd7, 0x9c, 0xd7, 0x9e, 0xd7,
    0xa2, 0xd7, 0x91, 0xd7, 0x93, 0x20, 0xd7, 0x9b,
    0xd7, 0xa8, 0xd7, 0xa2, 0xd7, 0x95, 0xd7, 0xaa,
    0x20, 0xd7, 0x90, 0xd7, 0x9c, 0xd7, 0x94, 0xd7,
    0x9b, 0xd7, 0x9d, 0x20, 0xd7, 0xaa, 0xd7, 0xa2,
    0xd7, 0x91, 0xd7, 0x93, 0xd7, 0x95, 0xd7, 0x9f,
    
    # Bitcoin block hash 0000000000000000000b1f8e1c17b0133d439174e52efbb0c41c3583a8aa66b0
    0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
    0x00, 0x0b, 0x1f, 0x8e, 0x1c, 0x17, 0xb0, 0x13,
    0x3d, 0x43, 0x91, 0x74, 0xe5, 0x2e, 0xfb, 0xb0,
    0xc4, 0x1c, 0x35, 0x83, 0xa8, 0xaa, 0x66, 0xb0,
    
    # Checkpoint block hash 0fca37ca667c2d550a6c4416dad9717e50927128c424fa4edbebc436ab13aeef
    0x0f, 0xca, 0x37, 0xca, 0x66, 0x7c, 0x2d, 0x55,
    0x0a, 0x6c, 0x44, 0x16, 0xda, 0xd9, 0x71, 0x7e,
    0x50, 0x92, 0x71, 0x28, 0xc4, 0x24, 0xfa, 0x4e,
    0xdb, 0xeb, 0xc4, 0x36, 0xab, 0x13, 0xae, 0xef,
])


# Bitcoin explorer link: 
# https://blockstream.info/block/0000000000000000000b1f8e1c17b0133d439174e52efbb0c41c3583a8aa66b0
# 
# Kaspad version release: 
# https://github.com/kaspanet/kaspad/releases/tag/v0.11.5-2 
assert(bytes.fromhex('0000000000000000000b1f8e1c17b0133d439174e52efbb0c41c3583a8aa66b0') == 
       bytes([
           0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00,
           0x00, 0x0b, 0x1f, 0x8e, 0x1c, 0x17, 0xb0, 0x13,
           0x3d, 0x43, 0x91, 0x74, 0xe5, 0x2e, 0xfb, 0xb0,
           0xc4, 0x1c, 0x35, 0x83, 0xa8, 0xaa, 0x66, 0xb0,]))

assert(bytes.fromhex('0fca37ca667c2d550a6c4416dad9717e50927128c424fa4edbebc436ab13aeef') == 
       bytes([
           0x0f, 0xca, 0x37, 0xca, 0x66, 0x7c, 0x2d, 0x55,
           0x0a, 0x6c, 0x44, 0x16, 0xda, 0xd9, 0x71, 0x7e,
           0x50, 0x92, 0x71, 0x28, 0xc4, 0x24, 0xfa, 0x4e,
           0xdb, 0xeb, 0xc4, 0x36, 0xab, 0x13, 0xae, 0xef,]))


In [6]:
# Build genesis's coinbase tx. 
# Golang ref: 
# https://github.com/kaspanet/kaspad/blob/master/domain/dagconfig/genesis.go#L51

genesis_coinbase_tx = type('Transaction', (object,), {})()
genesis_coinbase_tx.version = 0
genesis_coinbase_tx.subnetworkID = type('SubnetworkId', (object,), {})()
genesis_coinbase_tx.subnetworkID.subnetworkId = bytes.fromhex(
    '0100000000000000000000000000000000000000')
genesis_coinbase_tx.inputs = []
genesis_coinbase_tx.outputs = []
genesis_coinbase_tx.lockTime = 0
genesis_coinbase_tx.gas = 0
genesis_coinbase_tx.payload = genesis_tx_payload

In [7]:
pre_checkpoint_store = Store(r'D:\kaspad-data\kaspa-data-22-11-21-correct-utxo-commit\data')
current_store = Store(os.getenv('localappdata') + r'\Kaspad\kaspa-mainnet\datadir2')

In [8]:
genesis_header = current_store.get_raw_header(genesis_hash)

# Assert the genesis hash is correct
assert(header_hash(genesis_header) == genesis_hash)

In [9]:
# This shows that indeed current genesis refrences the checkpoint via the coinbase tx payload
assert(
    transaction_hash(genesis_coinbase_tx) == genesis_header.hashMerkleRoot.hash)

print(transaction_hash(genesis_coinbase_tx).hex()) 
print(genesis_header.hashMerkleRoot.hash.hex())

8ec898568c6801d13df4ee6e2a1b54b7e6236f671f20954f05306410518eeb32
8ec898568c6801d13df4ee6e2a1b54b7e6236f671f20954f05306410518eeb32


In [10]:
# Show that tips from current database link to genesis
tips, hst = current_store.tips()
assert_cryptographic_hash_chain_to_genesis(current_store, tips[0], genesis_hash)

Reached the queried genesis block: 
 58c2d4199e21f910d1571d114969cecef48f09f934d42ccb6a281a15868f2999 via 209 pruning points


In [11]:
# Now we move to the pre-halt database and show that the checkpoint was mined
# over the original genesis (with an empty UTXO-set commitment)

checkpoint_hash = bytes.fromhex(
    '0fca37ca667c2d550a6c4416dad9717e50927128c424fa4edbebc436ab13aeef')
checkpoint_header = pre_checkpoint_store.get_raw_header(checkpoint_hash)

# Assert the checkpoint hash is correct
assert(header_hash(checkpoint_header) == checkpoint_hash)

# Show that genesis and the checkpoint share the same UTXO commitment
assert(genesis_header.utxoCommitment.hash == checkpoint_header.utxoCommitment.hash)

print(genesis_header.utxoCommitment.hash.hex()) 
print(checkpoint_header.utxoCommitment.hash.hex())

710f27df423e63aa6cdb72b89ea5a06cffa399d66f167704455b5af59def8e20
710f27df423e63aa6cdb72b89ea5a06cffa399d66f167704455b5af59def8e20


In [12]:
# Golang ref from initial mainnet version: 
# https://github.com/kaspanet/kaspad/blob/v0.11.0/domain/dagconfig/genesis.go#L53C2-L56C49
original_genesis = bytes([
    0xca, 0xeb, 0x97, 0x96, 0x0a, 0x16, 0x0c, 0x21,
    0x1a, 0x6b, 0x21, 0x96, 0xbd, 0x78, 0x39, 0x9f,
    0xd4, 0xc4, 0xcc, 0x5b, 0x50, 0x9f, 0x55, 0xc1,
    0x2c, 0x8a, 0x7d, 0x81, 0x5f, 0x75, 0x36, 0xea,])

In [13]:
assert_cryptographic_hash_chain_to_genesis(
    pre_checkpoint_store, 
    checkpoint_hash, 
    original_genesis)

Reached the queried genesis block: 
 caeb97960a160c211a6b2196bd78399fd4c4cc5b509f55c12c8a7d815f7536ea via 5 pruning points


In [14]:
# Show that original genesis has an empty UTXO-set commitment

original_genesis_header = pre_checkpoint_store.get_raw_header(original_genesis)
assert(header_hash(original_genesis_header) == original_genesis)

# Golang ref: https://github.com/kaspanet/go-muhash/blob/main/muhash.go#L32
empty_muhash_hash = bytes([
    0x54, 0x4e, 0xb3, 0x14, 0x2c, 0x0, 0xf, 0xa, 
    0xd2, 0xc7, 0x6a, 0xc4, 0x1f, 0x42, 0x22, 0xab, 
    0xba, 0xba, 0xbe, 0xd8, 0x30, 0xee, 0xaf, 0xee, 
    0x4b, 0x6d, 0xc5, 0x6b, 0x52, 0xd5, 0xca, 0xc0])

assert(original_genesis_header.utxoCommitment.hash == empty_muhash_hash)

print(original_genesis_header.utxoCommitment.hash.hex()) 
print(empty_muhash_hash.hex())

544eb3142c000f0ad2c76ac41f4222abbababed830eeafee4b6dc56b52d5cac0
544eb3142c000f0ad2c76ac41f4222abbababed830eeafee4b6dc56b52d5cac0
